[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/philogicae/ai-notebooks-colab/blob/master/stable_diffusion_webui.ipynb)

# **Stable Diffusion WebUI Colab by [Philogicae](https://github.com/philogicae)**

- This notebook takes around **5min** to launch **Stable Diffusion** and generate a **public link** using Gradio (usable as webUI or to paste into the Photoshop extension).
- At launch, it will ask you for permission to connect to Google Drive (to load/save your config, output images and access your downloaded models).
- Feel free to add/remove your own **extensions**, **scripts** or **embeddings** below (Many on [Github](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Extensions) & [HuggingFace](https://huggingface.co/)).
- You can activate/deactivate the default **models** you want to load at startup, or use the [civitai](https://civitai.com/) extension,
but I strongly advise you to download your favorite models (using **Tools**) to keep them into your **Google Drive folder**: *stable-diffusion/models*
- To learn how to use the webUI -> Youtube/Discord
- To use it inside **Photoshop**, download and install the .ccx: [stable.art](https://github.com/isekaidev/stable.art#how-to-install)
- If you use a cracked version of Ps, to install the .ccx you will need: [UXP Developer Tools](https://developer.adobe.com/photoshop/uxp/devtool/installation/)
- For questions/bugs, drop me a message on [Twitter](https://twitter.com/philogicae)/[Github](https://github.com/philogicae)/[Telegram](https://t.me/philogicae).

# Tools

In [ ]:
#@markdown **Run this block to download a model from an URL directly to your Google drive:**
import os
from google.colab import drive # type: ignore
drive_path = "/content/drive"
data_dir = "/content/drive/MyDrive/stable-diffusion"

# Mount Google Drive
if not os.path.exists(drive_path):
    try:
        drive.mount(drive_path, force_remount=True)
        os.makedirs(data_dir, exist_ok=True)
    except:
        raise("Error mounting drive or with path variables")

# Download models directly into your Google Drive folder: stable-diffusion/models
models_path = f"{data_dir}/models"
os.makedirs(models_path, exist_ok=True)
%cd {models_path}

model_url = "" #@param {type:"string"}
if model_url:
    print(f"Downloading model from {model_url}")
    !wget --content-disposition {model_url}
    print(f"Downloaded model to {models_path}")
else:
    print("Error: Invalid infos")


# Setup & Run

In [ ]:
#@markdown **Drive settings**
import os
from google.colab import drive # type: ignore
drive_path = "/content/drive"
data_dir = "/content/drive/MyDrive/stable-diffusion" #@param {type:"string"}
sd_dir = "/content/stable-diffusion-webui"
%cd "/content"

### Mount Google Drive
if not os.path.exists(drive_path):
    try:
        drive.mount(drive_path, force_remount=True)
        os.makedirs(data_dir, exist_ok=True)
    except:
        raise("Error mounting drive or with path variables")

### Requirements
%pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
%pip install -q --pre triton
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

#@markdown **Extensions**
%cd {sd_dir}/extensions
images_browser = True #@param {type:"boolean"}
if images_browser:
    !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser.git
prompt_generator = True #@param {type:"boolean"}
if prompt_generator:
    !git clone https://github.com/imrayya/stable-diffusion-webui-Prompt_Generator.git
civitai_browser = True #@param {type:"boolean"}
if civitai_browser:
    !git clone https://github.com/philogicae/sd-civitai-browser.git
deforum = True #@param {type:"boolean"}
if deforum:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui.git
openOutpaint = False #@param {type:"boolean"}
if openOutpaint:
    !git clone https://github.com/zero01101/openOutpaint-webUI-extension.git

#@markdown **Scripts**
%cd {sd_dir}/scripts
run_n_times = True #@param {type:"boolean"}
if run_n_times:
    !wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py

#@markdown **Embeddings**
%cd {sd_dir}/embeddings
dblx = True #@param {type:"boolean"}
if dblx:
    !wget https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt

### Outputs / Config -> Saved to Google Drive
%cd {sd_dir}
output_path = f"{data_dir}/outputs"
os.makedirs(output_path, exist_ok=True)
!rm -Rf outputs && ln -s {output_path} outputs

config_path = f"{data_dir}/config"
os.makedirs(config_path, exist_ok=True)
for filename in ["config.json", "ui-config.json", "styles.csv", "artists.csv"]:
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            !touch {config_path}/{filename}
    !rm -f {filename} && ln -s {config_path}/{filename} {filename}

#@markdown **Default Models**
%cd {sd_dir}/models/Stable-diffusion
stable_diffusion_2_1_768x768 = True #@param {type:"boolean"}
if stable_diffusion_2_1_768x768:
    !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors
    !wget https://huggingface.co/webui/stable-diffusion-2-1/raw/main/v2-1_768-ema-pruned.yaml
stable_diffusion_inpainting_512x512 = False #@param {type:"boolean"}
if stable_diffusion_inpainting_512x512:
    !wget https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/512-inpainting-ema.safetensors
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inpainting-inference.yaml -O 512-inpainting-ema.yaml
stable_diffusion_upscaler_512x512 = False #@param {type:"boolean"}
if stable_diffusion_upscaler_512x512:
    !wget https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler/resolve/main/x4-upscaler-ema.safetensors
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/x4-upscaling.yaml -O x4-upscaler-ema.yaml
instruct_pix2pix_512x512 = False #@param {type:"boolean"}
if instruct_pix2pix_512x512:
    !wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.safetensors
stable_diffusion_1_5_512x512 = False #@param {type:"boolean"}
if stable_diffusion_1_5_512x512:
    !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors
    !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/v1-inference.yaml -O v1-5-pruned-emaonly.yaml

### Load Saved Models (From GoogleDrive/stable-diffusion/models)
models_path = f"{data_dir}/models"
os.makedirs(models_path, exist_ok=True)
for filename in os.listdir(models_path):
    !ln -s {models_path}/{filename} {filename}

#@markdown **[ControlNet](https://stable-diffusion-art.com/controlnet/) (To use with SD v1.5 models)**
canny = False #@param {type:"boolean"}
depth = False #@param {type:"boolean"}
hed = False #@param {type:"boolean"}
mlsd = False #@param {type:"boolean"}
normal = False #@param {type:"boolean"}
openpose = False #@param {type:"boolean"}
scribble = False #@param {type:"boolean"}
seg = False #@param {type:"boolean"}
if canny or depth or hed or mlsd or normal or openpose or scribble or seg:
    %cd {sd_dir}/extensions
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd {sd_dir}/extensions/sd-webui-controlnet/models
    if canny:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors
    if depth:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors
    if hed:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors
    if mlsd:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors
    if normal:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors
    if openpose:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors
    if scribble:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors
    if seg:
        !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors

### Launch
%cd {sd_dir}
!python launch.py --theme dark --share --xformers --medvram --api --enable-insecure-extension-access